In [ ]:
!nvidia-smi

# Init

In [35]:
!git clone --depth=1 https://github.com/AdeelH/potsdam-batch-exp.git

Cloning into 'potsdam-batch-exp'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 16 (delta 2), reused 14 (delta 1), pack-reused 0
Unpacking objects: 100% (16/16), done.


In [2]:
BATCH_SIZE = 16
PRETRAINED = False

MODEL_ARCH = 'deeplab'
MODEL_BASE = 'resnet101'
CHANNEL_VARIATION = 'rgb'
MODEL_VARIATION = f''

In [3]:
EXPERIMENT_NAME = f'ss_{CHANNEL_VARIATION}_{MODEL_ARCH}_{MODEL_BASE}{"p" if PRETRAINED else ""}'
print(EXPERIMENT_NAME)

S3_BUCKET = 'raster-vision-ahassan'
S3_ROOT = f'potsdam/experiments/output/{EXPERIMENT_NAME}'

ss_rgb_deeplab_resnet101


In [4]:
EPOCHS = 80
LR_START = 1e-1
LR_END = 1e-3

MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4

CHECKPOINT_INTERVAL = 10
BATCH_CB_INTERVAL = 10

In [5]:
import os
from pathlib import Path
import glob
from datetime import datetime

import numpy as np
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
import torch.utils
import torchvision as tv
from torchvision import transforms as tf

from fastai.vision import *
from fastai.metrics import error_rate

import matplotlib.pyplot as plt
%reload_ext autoreload
%autoreload 2
# %matplotlib inline

In [36]:
import sys
sys.path.append('potsdam-batch-exp/')

from utils_ import *
from io_ import *
from data_ import *
from transforms import *
from models import *
from training import *
from visualizations import *
from monitoring import *

In [7]:
CHANNELS = [ch_R, ch_G, ch_B]
CHIP_SIZE = 400
STRIDE = 200
DOWNSAMPLING = 2

TRAIN_SPLIT = 0.85

In [8]:
CLASS_NAMES = ['building', 'tree', 'low-vegetation', 'clutter', 'car', 'pavement']
NCLASSES = len(CLASS_NAMES)

In [ ]:
io_handler = S3IoHandler(
    local_root=EXPERIMENT_NAME, 
    s3_bucket=S3_BUCKET, 
    s3_root=S3_ROOT
)

# Data

In [ ]:
potsdam_dict = io_handler.load_pickled_file('potsdam/data/potsdam.pkl', 'data/potsdam.pkl')

## Prepare datasets

In [ ]:
train_transform, val_transform, x_transform, y_transform = tfs_potsdam(channels=CHANNELS, downsampling=DOWNSAMPLING)

In [ ]:
original_ds = Potsdam(potsdam_dict, chip_size=CHIP_SIZE, stride=STRIDE, tf=val_transform)
train_ds    = Potsdam(potsdam_dict, chip_size=CHIP_SIZE, stride=STRIDE, tf=train_transform, x_tf=x_transform, y_tf=y_transform)
val_ds      = Potsdam(potsdam_dict, chip_size=CHIP_SIZE, stride=STRIDE, tf=val_transform  , x_tf=x_transform, y_tf=y_transform)

### Train/val split

In [ ]:
train_split_size = int((len(train_ds) * TRAIN_SPLIT) // 1)
val_split_size = len(train_ds) - train_split_size
train_split_size, val_split_size

print('train_split_size', train_split_size)
print('val_split_size', val_split_size)

inds = np.arange(len(train_ds))

### Samplers

In [ ]:
train_sampler = torch.utils.data.SubsetRandomSampler(inds[:train_split_size])
val_sampler = torch.utils.data.SubsetRandomSampler(inds[train_split_size:])

assert len(set(train_sampler.indices) & set(val_sampler.indices)) == 0

# Model

Use FastAI to create a UNet from a Resnet18

In [37]:
model = get_deeplab_custom(NCLASSES, in_channels=3, pretrained=PRETRAINED, pretrained_backbone=PRETRAINED).cuda()

In [30]:
# # freeze(model.m.backbone)
# freeze(model.m.backbone.conv1)
# freeze(model.m.backbone.bn1)

# Train

## Training monitoring callbacks

In [ ]:

def get_epoch_monitor(io_handler, chkpt_interval=1, viz_root='visualizations/per_epoch'):
    assert chkpt_interval > 0

    def _monitor(model, optimizer, sched, logs):
        epoch = logs['epoch'][-1]

        track_best_model(io_handler, model, logs)

        if epoch % chkpt_interval == 0:
            io_handler.save_checkpoint(model, optimizer, sched, f'checkpoints/epoch_%04d' % (epoch), info=logs)

        log_str = logs_to_str(logs)
        print(log_str)

        io_handler.save_log('logs.pkl', logs)
        io_handler.save_log_str(f'logs.txt', log_str)
        
        make_plots(io_handler, logs)
        
    return _monitor

def get_batch_monitor(io_handler, viz_root='visualizations/per_batch', interval=4):

    def _monitor(model, epoch, batch_idx, batch, labels):
        pass

    return _monitor


In [ ]:
epoch_callback = get_epoch_monitor(io_handler, chkpt_interval=CHECKPOINT_INTERVAL)
batch_callback = get_batch_monitor(io_handler, interval=BATCH_CB_INTERVAL)

In [ ]:
train_params = {}
train_params['batch_size'] = BATCH_SIZE
train_params['val_batch_size'] = BATCH_SIZE

In [ ]:
train_dl = torch.utils.data.DataLoader(train_ds, sampler=train_sampler, batch_size=train_params['batch_size']    , pin_memory=False)
val_dl   = torch.utils.data.DataLoader(val_ds  , sampler=val_sampler  , batch_size=train_params['val_batch_size'], pin_memory=False)

In [ ]:
train_params = {}
train_params['epochs'] = EPOCHS
train_params['learning_rate'] = LR_START
train_params['learning_rate_min'] = LR_END

optimizer = optim.SGD(model.parameters(), lr=train_params['learning_rate'], momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
sched = optim.lr_scheduler.CosineAnnealingLR(optimizer, train_params['epochs'], eta_min=train_params['learning_rate_min'])


In [ ]:
from collections import defaultdict

if io_handler.checkpoint_exists():
    logs = restore_training_state(io_handler, model, optimizer, sched, train_params)
else:
    logs = defaultdict(list)

In [ ]:
plt.ioff()
train_seg(model, train_dl, val_dl, optimizer, sched, train_params, 
          epoch_callback=epoch_callback, batch_callback=batch_callback, logs=logs)

# Create animations

In [ ]:
# !apt update -y
# !apt install ffmpeg -y

In [ ]:
# !ffmpeg -framerate 30 -pattern_type glob -i "$EXPERIMENT_NAME/visualizations/per_batch/conv/*.png" -c:v libx264 \
#     -pix_fmt yuv420p "$EXPERIMENT_NAME/visualizations/per_batch/conv/conv.mp4" -y;
# !ffmpeg -framerate 30 -pattern_type glob -i "$EXPERIMENT_NAME/visualizations/per_batch/conv_grad/*.png" -c:v libx264 \
#     -pix_fmt yuv420p "$EXPERIMENT_NAME/visualizations/per_batch/conv_grad/conv_grad.mp4" -y;

In [ ]:
# io_handler.upload_file('visualizations/per_batch/conv/conv.mp4', 'visualizations/per_batch/conv.mp4')
# io_handler.upload_file('visualizations/per_batch/conv_grad/conv_grad.mp4', 'visualizations/per_batch/conv_grad.mp4')